## IMPORT Thư Viện


In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

## Import Data, Test

In [28]:
df_train_raw = pd.read_csv('../data/train.csv')
df_test_raw = pd.read_csv('../data/train.csv')

In [29]:
train_ID = df_train_raw['Id']
test_ID = df_test_raw['Id']
y_train_target = df_train_raw['SalePrice']
N_train = len(df_train_raw)

In [30]:
df_train_raw.drop(['Id', 'SalePrice'], axis=1, inplace=True)
df_test_raw.drop('Id', axis=1, inplace=True)

In [31]:
df_combined = pd.concat([df_train_raw, df_test_raw], ignore_index=True)

print(f"Kích thước dữ liệu hợp nhất: {df_combined.shape}")

Kích thước dữ liệu hợp nhất: (2920, 80)


In [32]:
# --- 1. Xử lý NaN bằng 'None' (Biến Phân loại) --- 
# Các cột mà NaN nghĩa là 'Không có' (theo logic Ex2)
cols_none = ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageType', 'GarageFinish',
             'GarageQual', 'GarageCond', 'MasVnrType', 'BsmtExposure',
             'BsmtFinType1', 'BsmtFinType2', 'Alley', 'PoolQC', 'Fence', 'MiscFeature']

for col in cols_none:
    if col in df_combined.columns:
        df_combined[col] = df_combined[col].fillna('None')

# --- 2. Xử lý NaN bằng Mean/0 (Biến Số) --- 
# LotFrontage dùng Mean
df_combined['LotFrontage'] = df_combined['LotFrontage'].fillna(df_combined['LotFrontage'].mean())

# Các cột diện tích/số lượng dùng 0
cols_zero = ['MasVnrArea', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
             'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
for col in cols_zero:
    if col in df_combined.columns:
        df_combined[col] = df_combined[col].fillna(0)

# --- 3. Xử lý các cột phân loại còn lại bằng Mode --- 
for col in ('MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'SaleType', 
            'Electrical', 'KitchenQual', 'Functional'):
    if col in df_combined.columns:
        df_combined[col] = df_combined[col].fillna(df_combined[col].mode()[0])

# --- 4. Xóa các cột không cần thiết ---
# (Giữ logic cũ của bạn)
df_combined.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)

print("Đã hoàn thành xử lý dữ liệu thiếu và xóa cột.")

Đã hoàn thành xử lý dữ liệu thiếu và xóa cột.


In [33]:
# --- 1. Mã hóa biến Phân loại (One-Hot Encoding) ---
df_encoded = pd.get_dummies(df_combined, drop_first=True)

# --- 2. Tách dữ liệu ra thành Train và Test --- 
X_train = df_encoded[:N_train] # Features huấn luyện (1460 mẫu)
X_test = df_encoded[N_train:]  # Features kiểm tra (1459 mẫu)

# --- 3. Chuẩn hóa Dữ liệu (Standard Scaling) ---
scaler = StandardScaler()

# FIT (học tham số) CHỈ trên X_train
scaler.fit(X_train)

# Transform trên X_train và X_test
X_train_scaled = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print(f"Kích thước X_train_scaled: {X_train_scaled.shape}")
print(f"Kích thước X_test_scaled: {X_test_scaled.shape}")

Kích thước X_train_scaled: (1460, 247)
Kích thước X_test_scaled: (1460, 247)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [36]:
# --- Tạo thư mục lưu trữ nếu chưa có ---
save_dir = './exps/feature_Labels' 
os.makedirs(save_dir, exist_ok=True)

# --- Lưu Dữ liệu Hoàn thiện ---
# 1. Lưu tập huấn luyện Features đã chuẩn hóa
X_train_scaled.to_csv(f'{save_dir}/ex2_train.csv', index=False)

# 2. Lưu tập kiểm tra Features đã chuẩn hóa
X_test_scaled.to_csv(f'{save_dir}/ex2_test.csv', index=False)

# 3. Lưu Target huấn luyện
y_train_target.to_csv(f'{save_dir}/y_train.csv', index=False, header=True)

print(f"\n✅ Hoàn thành tiền xử lý và lưu 3 file vào thư mục {save_dir}/")


✅ Hoàn thành tiền xử lý và lưu 3 file vào thư mục ./exps/feature_Labels/


ValueError: Input X contains NaN.
Ridge does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [39]:
prediction = model.predict(X_test_scaled) 

print(f"Số lượng dự đoán tạo ra: {len(prediction)}")

NameError: name 'model' is not defined

In [38]:
# --- GIẢ SỬ: Bạn đã có biến 'model' đã được huấn luyện ---
# Ví dụ: from sklearn.linear_model import Ridge; model = Ridge(alpha=10); model.fit(X_train_scaled, y_train_target)

# Thay thế dòng dưới bằng mô hình đã huấn luyện của bạn
# Ví dụ:
# prediction = model.predict(X_test_scaled) 

# Để code chạy được, tôi tạo prediction mẫu (chỉ dùng cho mục đích minh họa cú pháp submission)
prediction = np.random.rand(len(X_test_scaled)) * 100000 + 150000 
# Kích thước prediction phải là 1459

# --- Tạo DataFrame Submission (Sử dụng test_ID gốc 1459 mẫu) ---
test = pd.read_csv('../data/test.csv')
my_submission = pd.DataFrame({
    "Id": test['Id'], # Lấy 1459 Ids gốc
    "SalePrice": prediction # 1459 dự đoán tương ứng
})

# Lưu file Submission
submission_dir = '../submission'
os.makedirs(submission_dir, exist_ok=True)

my_submission.to_csv(f'{submission_dir}/exp2_submission.csv', index=False)

print(f"Đã tạo file submission mẫu: {submission_dir}/exp2_submission.csv (Kích thước: {my_submission.shape})")

ValueError: array length 1460 does not match index length 1459